#### Grab Data from API

Weather Data

https://openweathermap.org/current  
    
    
    API call: string ='api.openweathermap.org/data/2.5/weather?q={city name}'

In [1]:
import pymongo, requests, pprint, json, datetime, time
from pymongo import MongoClient

client=pymongo.MongoClient('pythonforbigdata_this_mongo_1')

In [24]:
from secrets import *

In [30]:
from secret import *
print(key)

349f8bfee059717073b02c2c6497f938


In [3]:
client.database_names()

['admin', 'config', 'local']

In [4]:
db = client.weatherExample

In [5]:
weatherCollection=db.weatherExample

## Get list of cities

***skipped downloading file, did this in prior notebook, see video 3._ for instructions and code.

In [13]:
#download bulk file using wget
!wget http://bulk.openweathermap.org/sample/city.list.json.gz

--2018-07-11 13:52:43--  http://bulk.openweathermap.org/sample/city.list.json.gz
Resolving bulk.openweathermap.org (bulk.openweathermap.org)... 78.46.48.103
Connecting to bulk.openweathermap.org (bulk.openweathermap.org)|78.46.48.103|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4292641 (4.1M) [application/octet-stream]
Saving to: ‘city.list.json.gz’

city.list.json.gz   100%[===================>]   4.09M   801KB/s    in 6.1s    

2018-07-11 13:52:49 (686 KB/s) - ‘city.list.json.gz’ saved [4292641/4292641]



In [14]:
#unzip file
!gunzip city.list.json.gz

In [12]:
#take a look at file
!head city.list.json

[
  {
    "id": 707860,
    "name": "Hurzuf",
    "country": "UA",
    "coord": {
      "lon": 34.283333,
      "lat": 44.549999
    }
  },


In [7]:
!pwd

/notebooks/ptyhonVideoForBigDataFiles/alexFiles


In [23]:
!ls

 city.list.json   secrets.py	     WeatherExample_video3.ipynb
 __pycache__	 'Video 3._.ipynb'


### get a list of cities provided by weather api 

In [9]:
with open('city.list.json','r') as inFile:
    citiesJson=json.loads(inFile.read())

In [10]:
citiesJson[0]

{'id': 707860,
 'name': 'Hurzuf',
 'country': 'UA',
 'coord': {'lon': 34.283333, 'lat': 44.549999}}

#### fuction to get multiple cities

In [13]:
#fuction to get multiple cities
def getData(city):
    requestString = 'https://api.openweathermap.org/data/2.5/weather?q={}&APPID={}'.format(city, key)
    res = requests.get(requestString)
    return res

In [14]:
type(citiesJson)

list

### limit to only Chilean cities  
#You don't need to consume all of the memory to create a list up front, as long as the iterator returns the 
#same thing a list would during iteration. 
#https://stackoverflow.com/questions/12319025/filters-in-python3

In [15]:
#creates a python object that is a filter(pointer object) on whole citiesJson python object conserving memory
citiesJsonCL = list(filter(lambda x:x[u'country']==u'CL', citiesJson))

In [16]:
len(citiesJsonCL)

369

In [17]:
#extract just single item that is easier to pass in list map is another type of python pointer
cities = list(map(lambda x:x['name'],citiesJsonCL))
ids = list(map(lambda x:x['id'], citiesJsonCL))

In [31]:
#pass 1st city to get data
res = getData(str(cities[0]))

In [32]:
res.reason

'OK'

In [34]:
#this is running the get command in function for 1st city in list
res

<Response [200]>

In [35]:
pprint.pprint(res.json())

{'base': 'stations',
 'clouds': {'all': 90},
 'cod': 200,
 'coord': {'lat': -43.12, 'lon': -73.62},
 'dt': 1531486800,
 'id': 3874943,
 'main': {'humidity': 100,
          'pressure': 1011,
          'temp': 283.15,
          'temp_max': 283.15,
          'temp_min': 283.15},
 'name': 'Puerto Quellon',
 'sys': {'country': 'CL',
         'id': 4661,
         'message': 0.0044,
         'sunrise': 1531484552,
         'sunset': 1531517904,
         'type': 1},
 'visibility': 8000,
 'weather': [{'description': 'light intensity drizzle',
              'icon': '09d',
              'id': 300,
              'main': 'Drizzle'}],
 'wind': {'speed': 1.5}}


In [36]:
print(cities[0:10])

['Puerto Quellon', 'Nacimiento', 'El Tabo', 'Región del Libertador General Bernardo O’Higgins', 'Lo Moscoso', 'Barón', 'Región de Los Lagos', 'Las Animas', 'Puerto Montt', 'Las Condes']


### cycle through 1st 10 cities in list and post to collection

In [37]:
## cycle through the two lists grab the data for city and insert into mongo db

for i, name in zip(ids[0:10], cities[0:10]):
    
    res = getData(str(name))
    
    if not res.status_code==200:
        print('Error grabbing data for {}, reason {}'.format(name, res.reason))
        #print(res.reason)
    
    else:
        try:
              weatherCollection.insert_one(res.json())
        except e:
              print('Error inserting to DB'.format(e))
              print('City {}'.format(name))
              
    time.sleep(1)
    #so we don't overload api

In [38]:
for doc in weatherCollection.find():
    pprint.pprint(doc)

{'_id': ObjectId('5b48b78dcc04c700102c6db5'),
 'base': 'stations',
 'clouds': {'all': 90},
 'cod': 200,
 'coord': {'lat': -43.12, 'lon': -73.62},
 'dt': 1531486800,
 'id': 3874943,
 'main': {'humidity': 100,
          'pressure': 1011,
          'temp': 283.15,
          'temp_max': 283.15,
          'temp_min': 283.15},
 'name': 'Puerto Quellon',
 'sys': {'country': 'CL',
         'id': 4661,
         'message': 0.0044,
         'sunrise': 1531484552,
         'sunset': 1531517904,
         'type': 1},
 'visibility': 8000,
 'weather': [{'description': 'light intensity drizzle',
              'icon': '09d',
              'id': 300,
              'main': 'Drizzle'}],
 'wind': {'speed': 1.5}}
{'_id': ObjectId('5b48b78ecc04c700102c6db6'),
 'base': 'stations',
 'clouds': {'all': 75},
 'cod': 200,
 'coord': {'lat': -37.5, 'lon': -72.67},
 'dt': 1531490400,
 'id': 3879123,
 'main': {'humidity': 93,
          'pressure': 1018,
          'temp': 284.15,
          'temp_max': 284.15,
          

### add US cities

In [39]:
#creates a python object that is a filter(pointer object) on whole citiesJson python object conserving memory
citiesJsonUS = list(filter(lambda x:x[u'country']==u'US', citiesJson))
len(citiesJsonUS)

19972

In [40]:
#extract just single item that is easier to pass in list map is another type of python pointer
cities = list(map(lambda x:x['name'],citiesJsonUS))
ids = list(map(lambda x:x['id'], citiesJsonUS))

In [41]:
## cycle through the two lists grab the data for city and insert into mongo db

for i, name in zip(ids[0:1000], cities[0:1000]):
    
    res = getData(str(name))
    
    if not res.status_code==200:
        print('Error grabbing data for {}, reason {}'.format(name, res.reason))
        #print(res.reason)
    
    else:
        try:
              weatherCollection.insert_one(res.json())
        except e:
              print('Error inserting to DB'.format(e))
              print('City {}'.format(name))
              
    time.sleep(1)
    #so we don't overload api

Error grabbing data for Frederick, reason Not Found


KeyboardInterrupt: 

In [42]:
weatherCollection.count()

764

In [43]:
def getTimestamp(dt):
    return pd.datetime.fromtimestamp(dt)

In [44]:
def getDate(tstamp):
    dt=datetime.datetime.fromtimestamp(tstamp)
    return dt.strftime('%b %d - %H:%m')

# 3.3 Querying MongoDB for useful information

In [45]:
cur=weatherCollection.find()

In [46]:
sortedCur=cur.sort('main.temp')

In [47]:
for doc in sortedCur.limit(10):
    try:
        print(doc['name'])
        print(doc['main']['temp'])
    except:
        print('Error: missing name/temp')

Saint Andrews
270.565
Saint Andrews
270.565
Cook
271.15
Clifton
271.15
Texas
272.665
Jamestown
272.865
Orange
274.315
Paxson
277.715
West Juneau
278.85
Juneau
278.85


In [48]:
cur.count()

764

## Find the max recorded temperatures over time per station

In [49]:
pipeline=[]
pipeline.append({'$group':{'_id':'$name','maxTemp':{'$max':'$main.temp'}}})
# Group measurements by city name, extract maximum recorded tmep for each
pipeline.append({'$limit':10})
# Limit results to first 10 cities

In [50]:
cur=weatherCollection.aggregate(pipeline=pipeline)

In [51]:
for d in cur:
    print(d['_id'],d['maxTemp'])

San Carlos Park 303.68
Port Orange 297.99
Palm Coast 302.42
New Smyrna Beach 303.02
Leesburg 298.72
Estero 303.68
Pinellas County 303.04
Osceola County 303.77
District of Columbia 299.21
Van Buren 293.15


## Get datetime of max temp per station

In [52]:
pipeline=[]
pipeline.append({'$match':{'name':{'$exists':True}}})
# Filter out dirty rows
pipeline.append({'$sort':{'name':1,'main.temp':-1}})
# Sort by name (esc) and temperature (asc)
pipeline.append({'$group':{'_id':'$name','maxTemp':{'$first':'$main.temp'},'date':{'$first':'$dt'}}})
# Group by name, grab maximum temperature and date of temperature
# Sorted by temp so grab first document from each group
pipeline.append({'$limit':10})
# limit to first 10 cities

In [53]:
cur=weatherCollection.aggregate(pipeline=pipeline)

In [54]:
for d in cur:
    print(getDate(d['date']),d['_id'],d['maxTemp'])
    print('')

Jul 13 - 14:07 Yuma County 302.215

Jul 13 - 14:07 Young 289.715

Jul 13 - 14:07 Winslow 297.57

Jul 13 - 14:07 Winona 299.15

Jul 13 - 14:07 Winchester 294.88

Jul 13 - 14:07 Wilson County 299.26

Jul 13 - 13:07 Wild Island 301.15

Jul 13 - 13:07 West Juneau 278.85

Jul 13 - 14:07 West Chester 298.4

Jul 13 - 13:07 West Cameron 297.96



### saving scrape and write to mongo as py file that will be called when needed.

    this should be updated to include logging files for with it is called.  Follow logging tutorial in seperate project to get help.

In [55]:
%%writefile openweather_scrape.py

import pymongo, requests, pprint, json, datetime, time
from pymongo import MongoClient
from secrets import *  #saved secret from weather api website (strings)


client=pymongo.MongoClient('pythonforbigdata_this_mongo_1')

db=client.weatherExample  #database called packt
weatherCollection=db.weather  # a collection called weather within weathExample database


#fuction to get multiple cities
def getData(city):
    requestString = 'https://api.openweathermap.org/data/2.5/weather?q={}&APPID={}'.format(city, key)
    res = requests.get(requestString)
    return res


# ## Get lsit of cities

with open('city.list.json', 'r') as inFile:
    citiesJson = json.loads(inFile.read())


#creates a python object that is a filter(pointer object) on whole citiesJson python object conserving memory
citiesJsonCL = list(filter(lambda x:x[u'country']==u'CL', citiesJson))

#extract just single item that is easier to pass in list map is another type of python pointer
cities = list(map(lambda x:x['name'],citiesJsonCL))
ids = list(map(lambda x:x['id'], citiesJsonCL))

## cycle through the two lists grab the data for city and insert into mongo db

for i, name in zip(ids, cities):
    
    res = getData(str(name))
    
    if not res.status_code==200:
        print('Error grabbing data for {}, reason {}'.format(name, res.reason))
        #print(res.reason)
    
    else:
        try:
              weatherCollection.insert_one(res.json())
        except e:
              print('Error inserting to DB'.format(e))
              print('City {}'.format(name))
              
    time.sleep(1)

Writing openweather_scrape.py


In [56]:
weatherCollection.count()

764

In [67]:
#from openweather_scrape.py import *

In [ ]:
bookmark:
    https://www.safaribooksonline.com/videos/working-with-big/9781788839068/9781788839068-video3_3